In [1]:
# 从sklearn 调入所需要的包
# from sklearn import datasets
from sklearn.model_selection import train_test_split #数据分隔出训练集和验证集
import lightgbm as lgb
import numpy as np
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score, recall_score
import lightgbm as lgb

In [2]:
path='./'
train = pd.read_csv(path+'input/train_set.csv')
test = pd.read_csv(path+'input/test_set.csv')
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25317 entries, 0 to 25316
Data columns (total 18 columns):
ID           25317 non-null int64
age          25317 non-null int64
job          25317 non-null object
marital      25317 non-null object
education    25317 non-null object
default      25317 non-null object
balance      25317 non-null int64
housing      25317 non-null object
loan         25317 non-null object
contact      25317 non-null object
day          25317 non-null int64
month        25317 non-null object
duration     25317 non-null int64
campaign     25317 non-null int64
pdays        25317 non-null int64
previous     25317 non-null int64
poutcome     25317 non-null object
y            25317 non-null int64
dtypes: int64(9), object(9)
memory usage: 3.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10852 entries, 0 to 10851
Data columns (total 17 columns):
ID           10852 non-null int64
age          10852 non-null int64
job          10852 non-null object
mar

In [3]:
train.describe()

ID           age        balance           day      duration  \
count  25317.000000  25317.000000   25317.000000  25317.000000  25317.000000   
mean   12659.000000     40.935379    1357.555082     15.835289    257.732393   
std     7308.532719     10.634289    2999.822811      8.319480    256.975151   
min        1.000000     18.000000   -8019.000000      1.000000      0.000000   
25%     6330.000000     33.000000      73.000000      8.000000    103.000000   
50%    12659.000000     39.000000     448.000000     16.000000    181.000000   
75%    18988.000000     48.000000    1435.000000     21.000000    317.000000   
max    25317.000000     95.000000  102127.000000     31.000000   3881.000000   

           campaign         pdays      previous             y  
count  25317.000000  25317.000000  25317.000000  25317.000000  
mean       2.772050     40.248766      0.591737      0.116957  
std        3.136097    100.213541      2.568313      0.321375  
min        1.000000     -1.000000      0.000000      0.000000  
25%        1.000000     -1.000000      0.000000      0.000000  
50%        2.000000     -1.000000      0.000000      0.000000  
75%        3.000000     -1.000000      0.000000      0.000000  
max       55.000000    854.000000    275.000000      1.000000

In [4]:
test.describe()

ID           age       balance           day      duration  \
count  10852.000000  10852.000000  10852.000000  10852.000000  10852.000000   
mean   30743.500000     41.040638   1393.157298     15.680151    257.206137   
std     3132.846895     10.652369   3318.497101      8.302317    250.480906   
min    25318.000000     18.000000  -2604.000000      1.000000      0.000000   
25%    28030.750000     33.000000     72.000000      8.000000    102.000000   
50%    30743.500000     39.000000    450.000000     16.000000    181.000000   
75%    33456.250000     49.000000   1440.250000     21.000000    322.000000   
max    36169.000000     94.000000  81204.000000     31.000000   3102.000000   

           campaign         pdays      previous  
count  10852.000000  10852.000000  10852.000000  
mean       2.770365     39.954755      0.546443  
std        3.063481     99.524056      1.805938  
min        1.000000     -1.000000      0.000000  
25%        1.000000     -1.000000      0.000000  
50%        2.000000     -1.000000      0.000000  
75%        3.000000     -1.000000      0.000000  
max       58.000000    871.000000     51.000000

In [4]:
train.corr()

ID       age   balance       day  duration  campaign  \
ID        1.000000  0.008465  0.032719 -0.020171  0.229149 -0.038265   
age       0.008465  1.000000  0.093740 -0.016070  0.000416  0.006171   
balance   0.032719  0.093740  1.000000  0.010245  0.026042 -0.010419   
day      -0.020171 -0.016070  0.010245  1.000000 -0.031946  0.168830   
duration  0.229149  0.000416  0.026042 -0.031946  1.000000 -0.087780   
campaign -0.038265  0.006171 -0.010419  0.168830 -0.087780  1.000000   
pdays     0.065807 -0.026431  0.001032 -0.092892  0.000040 -0.089224   
previous  0.047028  0.006575  0.015792 -0.050706  0.001315 -0.031667   
y         0.556627  0.029916  0.057564 -0.031886  0.394746 -0.075173   

             pdays  previous         y  
ID        0.065807  0.047028  0.556627  
age      -0.026431  0.006575  0.029916  
balance   0.001032  0.015792  0.057564  
day      -0.092892 -0.050706 -0.031886  
duration  0.000040  0.001315  0.394746  
campaign -0.089224 -0.031667 -0.075173  
pdays     1.000000  0.411688  0.107565  
previous  0.411688  1.000000  0.088337  
y         0.107565  0.088337  1.000000

In [5]:
train.job.value_counts()

blue-collar      5456
management       5296
technician       4241
admin.           2909
services         2342
retired          1273
self-employed     884
entrepreneur      856
unemployed        701
housemaid         663
student           533
unknown           163
Name: job, dtype: int64

In [6]:
test['y']=-1
print(len(test.columns))

18


In [7]:
data = train.append(test).reset_index(drop=True)
# data = pd.concat([train,test]).rese_index(drop=True)
print(data.head())

   ID  age         job   marital  education default  balance housing loan  \
0   1   43  management   married   tertiary      no      291     yes   no   
1   2   42  technician  divorced    primary      no     5076     yes   no   
2   3   47      admin.   married  secondary      no      104     yes  yes   
3   4   28  management    single  secondary      no     -994     yes  yes   
4   5   42  technician  divorced  secondary      no     2974     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome  y  
0   unknown    9   may       150         2     -1         0  unknown  0  
1  cellular    7   apr        99         1    251         2    other  0  
2  cellular   14   jul        77         2     -1         0  unknown  0  
3  cellular   18   jul       174         2     -1         0  unknown  0  
4   unknown   21   may       187         5     -1         0  unknown  0  


In [8]:
path='./'
import os
import json
import gc
# os.system('pip install numba')
from numba import jit
#tqdm
# os.system('pip install tqdm')
from tqdm import tqdm_notebook
from tqdm import tqdm

#Integrated model
# os.system('pip install lightgbm')
import lightgbm as lgb
# os.system('pip install catboost==0.15.2')
import catboost as cbt
# os.system('pip install xgboost')
# import xgboost as xgb

#base import 
import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score
#about time
import time
import datetime 
from datetime import datetime, timedelta

#Garbage collection
import gc
# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist
#other
from collections import Counter 
from statistics import mode 
    #warning
import warnings
warnings.filterwarnings("ignore")
import json 
import math
from itertools import product
import ast 

In [9]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['ID','y']]
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
    data['count_' + i] = data.groupby([i])[i].transform('count')
    data[i] = lbl.fit_transform(data[i].astype(str))

In [10]:
data.head()

ID  age  job  marital  education  default  balance  housing  loan  contact  \
0   1   43    4        1          2        0      291        1     0        2   
1   2   42    9        0          0        0     5076        1     0        0   
2   3   47    0        1          1        0      104        1     1        0   
3   4   28    4        2          1        0     -994        1     1        0   
4   5   42    9        0          1        0     2974        1     0        2   

        ...        y  count_job  count_marital  count_education  \
0       ...        0       7595          21857            10654   
1       ...        0       6083           4140             5493   
2       ...        0       4124          21857            18509   
3       ...        0       7595          10172            18509   
4       ...        0       6083           4140            18509   

   count_default  count_housing  count_loan  count_contact  count_month  \
0          35524          19959       30363          10393        10957   
1          35524          19959       30363          23437         2365   
2          35524          19959        5806          23437         5569   
3          35524          19959        5806          23437         5569   
4          35524          19959       30363          10393        10957   

   count_poutcome  
0           29554  
1            1475  
2           29554  
3           29554  
4           29554  

[5 rows x 27 columns]

1ID Int客户唯一标识

2age Int客户年龄

3job String客户的职业

4marital String婚姻状况

5education String受教育水平

6default String是否有违约记录

7balance Int每年账户的平均余额

8housing String是否有住房贷款

9loan String是否有个人贷款

10contact String与客户联系的沟通方式

11day Int最后一次联系的时间（几号）

12month String最后一次联系的时间（月份）

13duration Int最后一次联系的交流时长

14campaign Int在本次活动中，与该客户交流过的次数

15pdays Int距离上次活动最后一次联系该客户，过去了多久（999表示没有联系过）16previousInt在本次活动之前，与该客户交流过的次数

17poutcome String上一次活动的结果

18y Int预测客户是否会订购定期存款业务

In [11]:
data.head()

ID  age  job  marital  education  default  balance  housing  loan  contact  \
0   1   43    4        1          2        0      291        1     0        2   
1   2   42    9        0          0        0     5076        1     0        0   
2   3   47    0        1          1        0      104        1     1        0   
3   4   28    4        2          1        0     -994        1     1        0   
4   5   42    9        0          1        0     2974        1     0        2   

        ...        y  count_job  count_marital  count_education  \
0       ...        0       7595          21857            10654   
1       ...        0       6083           4140             5493   
2       ...        0       4124          21857            18509   
3       ...        0       7595          10172            18509   
4       ...        0       6083           4140            18509   

   count_default  count_housing  count_loan  count_contact  count_month  \
0          35524          19959       30363          10393        10957   
1          35524          19959       30363          23437         2365   
2          35524          19959        5806          23437         5569   
3          35524          19959        5806          23437         5569   
4          35524          19959       30363          10393        10957   

   count_poutcome  
0           29554  
1            1475  
2           29554  
3           29554  
4           29554  

[5 rows x 27 columns]

In [12]:
feats = [i for i in data.columns if i not in ['ID','y']]
feats

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'count_job',
 'count_marital',
 'count_education',
 'count_default',
 'count_housing',
 'count_loan',
 'count_contact',
 'count_month',
 'count_poutcome']

In [1]:
import lightgbm as lgb
model = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'auc',
    subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.02, random_state=2017
    )
train_x =data[data['y']!=-1][feats]
train_y =data[data['y']!=-1]['y']
testx= data[data['y']==-1][feats]
# train_x, test_x, train_y, test_y = train_test_split(data1, label1, test_size=0.3, random_state=42)
model.fit(train_x,train_y)
test_pre = model.predict_proba(testx)[:,1]#, num_iteration=model.best_iteration

NameError: name 'data' is not defined

In [16]:
print(len(test_pre))

10852


In [17]:
pre=data[data['y']==-1][['ID']]
pre['pred']=test_pre
print(len(pre))

10852


In [18]:
pre.head()

ID      pred
25317  25318  0.034818
25318  25319  0.004100
25319  25320  0.001684
25320  25321  0.810440
25321  25322  0.014357

In [19]:
pre.to_csv('baseline2.csv',index=False )

0    1
1    2
2    3
3    4
4    5
Name: ID, dtype: object

In [14]:
#5折交叉验证
from sklearn.model_selection import KFold
n_splits=5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
train_x = data[data['y']!=-1][feats]
train_y = data[data['y']!=-1]['y']
res=data[data['y']==-1][['ID']]

test_x= data[data['y']==-1][feats]
res['pred'] = 0
for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    model.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(test_x1, test_y1)],eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += model.predict_proba(test_x)[:,1]

res['pred'] = res['pred']/n_splits
res.to_csv('./submission10_fold.csv', index=False)

[1]	training's auc: 0.772468	valid_1's auc: 0.77314
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.910633	valid_1's auc: 0.897851
[3]	training's auc: 0.909232	valid_1's auc: 0.894967
[4]	training's auc: 0.902539	valid_1's auc: 0.890017
[5]	training's auc: 0.917077	valid_1's auc: 0.902046
[6]	training's auc: 0.925447	valid_1's auc: 0.913987
[7]	training's auc: 0.925239	valid_1's auc: 0.912683
[8]	training's auc: 0.926928	valid_1's auc: 0.916865
[9]	training's auc: 0.926311	valid_1's auc: 0.91568
[10]	training's auc: 0.92501	valid_1's auc: 0.913934
[11]	training's auc: 0.92775	valid_1's auc: 0.916633
[12]	training's auc: 0.929284	valid_1's auc: 0.918103
[13]	training's auc: 0.929968	valid_1's auc: 0.918777
[14]	training's auc: 0.929286	valid_1's auc: 0.917748
[15]	training's auc: 0.929348	valid_1's auc: 0.917719
[16]	training's auc: 0.928625	valid_1's auc: 0.916171
[17]	training's auc: 0.929925	valid_1's auc: 0.91798
[18]	training's auc: 0.931175	val

[300]	training's auc: 0.961092	valid_1's auc: 0.932872
[301]	training's auc: 0.961147	valid_1's auc: 0.932902
[302]	training's auc: 0.9612	valid_1's auc: 0.932898
[303]	training's auc: 0.961281	valid_1's auc: 0.932883
[304]	training's auc: 0.961339	valid_1's auc: 0.932852
[305]	training's auc: 0.961406	valid_1's auc: 0.932844
[306]	training's auc: 0.961477	valid_1's auc: 0.932863
[307]	training's auc: 0.961553	valid_1's auc: 0.93288
[308]	training's auc: 0.961637	valid_1's auc: 0.932874
[309]	training's auc: 0.961681	valid_1's auc: 0.932897
[310]	training's auc: 0.961782	valid_1's auc: 0.933004
[311]	training's auc: 0.961864	valid_1's auc: 0.933059
[312]	training's auc: 0.961924	valid_1's auc: 0.933092
[313]	training's auc: 0.96203	valid_1's auc: 0.933098
[314]	training's auc: 0.962094	valid_1's auc: 0.933084
[315]	training's auc: 0.962145	valid_1's auc: 0.933051
[316]	training's auc: 0.962189	valid_1's auc: 0.933135
[317]	training's auc: 0.962254	valid_1's auc: 0.933128
[318]	training

[598]	training's auc: 0.975968	valid_1's auc: 0.933671
[599]	training's auc: 0.97602	valid_1's auc: 0.933702
[600]	training's auc: 0.976052	valid_1's auc: 0.933702
[601]	training's auc: 0.976085	valid_1's auc: 0.933733
[602]	training's auc: 0.976128	valid_1's auc: 0.93372
[603]	training's auc: 0.976174	valid_1's auc: 0.933664
[604]	training's auc: 0.976221	valid_1's auc: 0.933672
[605]	training's auc: 0.976274	valid_1's auc: 0.933708
[606]	training's auc: 0.976322	valid_1's auc: 0.9337
[607]	training's auc: 0.976356	valid_1's auc: 0.933705
[608]	training's auc: 0.976387	valid_1's auc: 0.933703
[609]	training's auc: 0.976417	valid_1's auc: 0.933695
[610]	training's auc: 0.976457	valid_1's auc: 0.933721
[611]	training's auc: 0.976493	valid_1's auc: 0.933762
[612]	training's auc: 0.976541	valid_1's auc: 0.933774
[613]	training's auc: 0.976571	valid_1's auc: 0.933764
[614]	training's auc: 0.976617	valid_1's auc: 0.933799
[615]	training's auc: 0.976661	valid_1's auc: 0.93383
[616]	training'

[269]	training's auc: 0.958397	valid_1's auc: 0.94135
[270]	training's auc: 0.958468	valid_1's auc: 0.941307
[271]	training's auc: 0.958526	valid_1's auc: 0.941296
[272]	training's auc: 0.958577	valid_1's auc: 0.941285
[273]	training's auc: 0.958663	valid_1's auc: 0.941364
[274]	training's auc: 0.958729	valid_1's auc: 0.941437
[275]	training's auc: 0.958792	valid_1's auc: 0.941418
[276]	training's auc: 0.958861	valid_1's auc: 0.941423
[277]	training's auc: 0.958932	valid_1's auc: 0.941442
[278]	training's auc: 0.958994	valid_1's auc: 0.941454
[279]	training's auc: 0.959042	valid_1's auc: 0.941454
[280]	training's auc: 0.95915	valid_1's auc: 0.941513
[281]	training's auc: 0.959237	valid_1's auc: 0.941487
[282]	training's auc: 0.959354	valid_1's auc: 0.941485
[283]	training's auc: 0.959429	valid_1's auc: 0.941496
[284]	training's auc: 0.959506	valid_1's auc: 0.941482
[285]	training's auc: 0.959573	valid_1's auc: 0.94149
[286]	training's auc: 0.959622	valid_1's auc: 0.94151
[287]	training

[120]	training's auc: 0.948272	valid_1's auc: 0.933597
[121]	training's auc: 0.948344	valid_1's auc: 0.933572
[122]	training's auc: 0.948405	valid_1's auc: 0.933701
[123]	training's auc: 0.948554	valid_1's auc: 0.933793
[124]	training's auc: 0.94863	valid_1's auc: 0.933844
[125]	training's auc: 0.948719	valid_1's auc: 0.933828
[126]	training's auc: 0.948778	valid_1's auc: 0.933857
[127]	training's auc: 0.948826	valid_1's auc: 0.933972
[128]	training's auc: 0.94892	valid_1's auc: 0.934108
[129]	training's auc: 0.949011	valid_1's auc: 0.93403
[130]	training's auc: 0.949115	valid_1's auc: 0.934056
[131]	training's auc: 0.94918	valid_1's auc: 0.934073
[132]	training's auc: 0.949243	valid_1's auc: 0.934058
[133]	training's auc: 0.949334	valid_1's auc: 0.934127
[134]	training's auc: 0.949366	valid_1's auc: 0.934129
[135]	training's auc: 0.949404	valid_1's auc: 0.934161
[136]	training's auc: 0.949502	valid_1's auc: 0.934233
[137]	training's auc: 0.949556	valid_1's auc: 0.93424
[138]	training'

...

[818]	training's auc: 0.982957	valid_1's auc: 0.936308
[819]	training's auc: 0.982978	valid_1's auc: 0.936321
[820]	training's auc: 0.983015	valid_1's auc: 0.93631
[821]	training's auc: 0.983037	valid_1's auc: 0.936335
[822]	training's auc: 0.983053	valid_1's auc: 0.936343
[823]	training's auc: 0.983092	valid_1's auc: 0.93628
[824]	training's auc: 0.983118	valid_1's auc: 0.936259
[825]	training's auc: 0.983143	valid_1's auc: 0.936265
[826]	training's auc: 0.983163	valid_1's auc: 0.936246
[827]	training's auc: 0.983183	valid_1's auc: 0.936251
[828]	training's auc: 0.983212	valid_1's auc: 0.936241
[829]	training's auc: 0.983241	valid_1's auc: 0.936224
[830]	training's auc: 0.983283	valid_1's auc: 0.936228
[831]	training's auc: 0.983311	valid_1's auc: 0.936211
[832]	training's auc: 0.983334	valid_1's auc: 0.936209
[833]	training's auc: 0.983365	valid_1's auc: 0.936201
[834]	training's auc: 0.983395	valid_1's auc: 0.936213
[835]	training's auc: 0.983414	valid_1's auc: 0.936198
[836]	traini

[193]	training's auc: 0.953312	valid_1's auc: 0.935297
[194]	training's auc: 0.953371	valid_1's auc: 0.935285
[195]	training's auc: 0.953441	valid_1's auc: 0.935303
[196]	training's auc: 0.953516	valid_1's auc: 0.935324
[197]	training's auc: 0.953585	valid_1's auc: 0.935331
[198]	training's auc: 0.95367	valid_1's auc: 0.935369
[199]	training's auc: 0.953716	valid_1's auc: 0.935391
[200]	training's auc: 0.953764	valid_1's auc: 0.935384
[201]	training's auc: 0.953831	valid_1's auc: 0.935437
[202]	training's auc: 0.953923	valid_1's auc: 0.935437
[203]	training's auc: 0.95402	valid_1's auc: 0.935454
[204]	training's auc: 0.954148	valid_1's auc: 0.935482
[205]	training's auc: 0.954227	valid_1's auc: 0.935411
[206]	training's auc: 0.954294	valid_1's auc: 0.935341
[207]	training's auc: 0.954359	valid_1's auc: 0.935407
[208]	training's auc: 0.954429	valid_1's auc: 0.935416
[209]	training's auc: 0.954558	valid_1's auc: 0.935534
[210]	training's auc: 0.954602	valid_1's auc: 0.93551
[211]	trainin

[491]	training's auc: 0.971228	valid_1's auc: 0.938608
[492]	training's auc: 0.97126	valid_1's auc: 0.938619
[493]	training's auc: 0.971307	valid_1's auc: 0.938648
[494]	training's auc: 0.97137	valid_1's auc: 0.938627
[495]	training's auc: 0.971406	valid_1's auc: 0.938647
[496]	training's auc: 0.971458	valid_1's auc: 0.938643
[497]	training's auc: 0.971499	valid_1's auc: 0.938637
[498]	training's auc: 0.971542	valid_1's auc: 0.938613
[499]	training's auc: 0.971587	valid_1's auc: 0.938618
[500]	training's auc: 0.971626	valid_1's auc: 0.938577
[501]	training's auc: 0.97166	valid_1's auc: 0.93856
[502]	training's auc: 0.971706	valid_1's auc: 0.938528
[503]	training's auc: 0.971735	valid_1's auc: 0.938505
[504]	training's auc: 0.971784	valid_1's auc: 0.938483
[505]	training's auc: 0.971811	valid_1's auc: 0.93851
[506]	training's auc: 0.971843	valid_1's auc: 0.938504
[507]	training's auc: 0.971887	valid_1's auc: 0.938515
[508]	training's auc: 0.971927	valid_1's auc: 0.93855
[509]	training's

[217]	training's auc: 0.956349	valid_1's auc: 0.923759
[218]	training's auc: 0.956415	valid_1's auc: 0.923724
[219]	training's auc: 0.956456	valid_1's auc: 0.923714
[220]	training's auc: 0.95652	valid_1's auc: 0.923694
[221]	training's auc: 0.956567	valid_1's auc: 0.923697
[222]	training's auc: 0.956613	valid_1's auc: 0.923713
[223]	training's auc: 0.956672	valid_1's auc: 0.923706
[224]	training's auc: 0.956763	valid_1's auc: 0.923739
[225]	training's auc: 0.956828	valid_1's auc: 0.923792
[226]	training's auc: 0.956882	valid_1's auc: 0.923828
[227]	training's auc: 0.95695	valid_1's auc: 0.923825
[228]	training's auc: 0.957049	valid_1's auc: 0.923829
[229]	training's auc: 0.957126	valid_1's auc: 0.923895
[230]	training's auc: 0.957246	valid_1's auc: 0.92399
[231]	training's auc: 0.957302	valid_1's auc: 0.923999
[232]	training's auc: 0.957352	valid_1's auc: 0.923935
[233]	training's auc: 0.957409	valid_1's auc: 0.923927
[234]	training's auc: 0.957468	valid_1's auc: 0.923884
[235]	trainin

[515]	training's auc: 0.973227	valid_1's auc: 0.924914
[516]	training's auc: 0.973266	valid_1's auc: 0.924889
[517]	training's auc: 0.973306	valid_1's auc: 0.924917
[518]	training's auc: 0.973347	valid_1's auc: 0.924914
[519]	training's auc: 0.973413	valid_1's auc: 0.924924
[520]	training's auc: 0.973455	valid_1's auc: 0.9249
[521]	training's auc: 0.973494	valid_1's auc: 0.924886
[522]	training's auc: 0.973542	valid_1's auc: 0.924895
[523]	training's auc: 0.973586	valid_1's auc: 0.924917
[524]	training's auc: 0.973633	valid_1's auc: 0.924904
[525]	training's auc: 0.973684	valid_1's auc: 0.92496
[526]	training's auc: 0.973727	valid_1's auc: 0.92497
[527]	training's auc: 0.973777	valid_1's auc: 0.924946
[528]	training's auc: 0.973837	valid_1's auc: 0.92498
[529]	training's auc: 0.973893	valid_1's auc: 0.924984
[530]	training's auc: 0.973936	valid_1's auc: 0.924942
[531]	training's auc: 0.973967	valid_1's auc: 0.924962
[532]	training's auc: 0.973991	valid_1's auc: 0.924951
[533]	training'